In [1]:
%load_ext autoreload
%autoreload 2

%pylab inline

import dataloaders
import models
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics 
import sklearn.cluster as cluster
import numpy as np 
import random
import classifiers
from torch.autograd import Variable
import torch
import torch.nn as nn 
import torch.optim as optim

from imblearn.under_sampling import RandomUnderSampler

Populating the interactive namespace from numpy and matplotlib


In [42]:
output_file_vot = '../data/bitcoinotc-cleaned.csv'
output_file_nam = '../data/bitcoinotc-cleaned.csv'
data = dataloaders.RegressionDataset(output_file_vot, output_file_nam, ratio=0.8, delimiter=',')

In [44]:
num_nodes = data.get_num_nodes()
dims = 32
epochs = 200
lr = 0.1
lr_decay=0.0
weight_decay=0.0
lam = 0.00055
X_train, y_train = data.get_training_set()
X_test, y_test = data.get_testing_set()
p = 2
X = data.X
y = data.y

model_fitter = models.fit_pseudo_kernel_model

In [45]:
kernel_model = model_fitter(num_nodes, dims, X_train, y_train, epochs=epochs, p=p, 
                                              lr=lr,lr_decay=lr_decay, lam=lam, weight_decay=weight_decay, undersample=True)

The loss at epoch  1  was  0.6944483518600464
The loss at epoch  2  was  0.70699143409729
The loss at epoch  3  was  0.6268907189369202
The loss at epoch  4  was  0.603489339351654
The loss at epoch  5  was  0.5837088227272034
The loss at epoch  6  was  0.5581854581832886
The loss at epoch  7  was  0.5141685009002686
The loss at epoch  8  was  0.46782389283180237
The loss at epoch  9  was  0.45148175954818726
The loss at epoch  10  was  0.45075052976608276
The loss at epoch  11  was  0.43370285630226135
The loss at epoch  12  was  0.42875003814697266
The loss at epoch  13  was  0.4158822298049927
The loss at epoch  14  was  0.4134719669818878
The loss at epoch  15  was  0.4125863313674927
The loss at epoch  16  was  0.4085146188735962
The loss at epoch  17  was  0.40044617652893066
The loss at epoch  18  was  0.3986360728740692
The loss at epoch  19  was  0.40023308992385864
The loss at epoch  20  was  0.39684152603149414
The loss at epoch  21  was  0.39647093415260315
The loss at epoc

The loss at epoch  174  was  0.36186888813972473
The loss at epoch  175  was  0.3595693111419678
The loss at epoch  176  was  0.36007922887802124
The loss at epoch  177  was  0.3633401393890381
The loss at epoch  178  was  0.36127105355262756
The loss at epoch  179  was  0.36141589283943176
The loss at epoch  180  was  0.3613925576210022
The loss at epoch  181  was  0.36235737800598145
The loss at epoch  182  was  0.3621450662612915
The loss at epoch  183  was  0.3646529018878937
The loss at epoch  184  was  0.35929709672927856
The loss at epoch  185  was  0.360311895608902
The loss at epoch  186  was  0.35956570506095886
The loss at epoch  187  was  0.36146101355552673
The loss at epoch  188  was  0.3622859716415405
The loss at epoch  189  was  0.36171743273735046
The loss at epoch  190  was  0.3606744408607483
The loss at epoch  191  was  0.3603665232658386
The loss at epoch  192  was  0.36017024517059326
The loss at epoch  193  was  0.3624941110610962
The loss at epoch  194  was  0.

In [71]:
method = 'concat'
X = []
y = []
for u, v in X_train:
    #print(u, v)
    X.append(kernel_model.get_edge_features(int(u), int(v), method))
    y.append(data.get_response(u, v))
    
X = np.array(X)
y = np.array(y)

In [72]:
reg = linear_model.LinearRegression()

In [73]:
reg.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [74]:
X1 = []
y1 = []
for u, v in X_test:
    #print(u, v)
    X1.append(kernel_model.get_edge_features(int(u), int(v), method))
    y1.append(data.get_response(u, v))
    
X1 = np.array(X1)
y1 = np.array(y1)

In [75]:
predictions = reg.predict(X1)

In [77]:
sqrt(metrics.mean_squared_error(y1, predictions))

0.32774757169952928

In [79]:
import scipy.stats as stats

In [80]:
stats.stats.pearsonr(y1, predictions)

(0.42764165696714906, 1.6539793037368739e-314)

In [81]:
z = y1 * predictions

In [83]:
len(z[z < 0])

1002

In [84]:
len(z[z >= 0])

6117